In [1]:
from dataloader import *

In [2]:
import matplotlib.pyplot as plt
import numpy as np
tmp = dataloader("../data/clover")
imu, rotor, mocap = tmp.runPipeline()
    

[dataloader.py]26::Dataloader Started
  0%|          | 0/1 [00:00<?, ?it/s][dataloader.py]34::Loading New folder


[INFO]  Data folder c:\Users\dhruv\Desktop\Penn\Sem2\ESE650\FinalProject\code\data\clover\yawConstant\New folder\rosbag already exists. Not creating.


100%|██████████| 1/1 [00:09<00:00,  9.90s/it]


In [3]:
x,y,z = mocap['pose.position.x'].to_numpy(),mocap['pose.position.y'].to_numpy(),mocap['pose.position.z'].to_numpy()


In [5]:
print(mocap['Time'][0]-imu['Time'][0])
print()
# imu['linear_acceleration.x']

-1.7363359928131104



In [4]:
from ahrs.filters import Madgwick, EKF
gyrox,gyroy,gyroz = imu['angular_velocity.x'].to_numpy(),imu['angular_velocity.y'].to_numpy(),imu['angular_velocity.z'].to_numpy()
accx,accy,accz = imu['linear_acceleration.x'].to_numpy(),imu['linear_acceleration.y'].to_numpy(),imu['linear_acceleration.z'].to_numpy()
# madgwick = Madgwick(gyr=, acc=acc_data)     

In [5]:
gyro = np.column_stack((np.column_stack((gyrox.reshape(-1,1),gyroy.reshape(-1,1))),gyroz.reshape(-1,1)))
acc = np.column_stack((np.column_stack((accx.reshape(-1,1),accy.reshape(-1,1))),accz.reshape(-1,1)))
mag = np.zeros_like(acc)
# mag[:,0] += 254.3
mag[:,1] += 1
# mag[:,2] += 558.6


In [8]:
madgwick = Madgwick(gyr=gyro, acc=acc, gain_imu=1e-100000)     

In [9]:
from scipy.spatial.transform import Rotation as R
R_w_ned = np.array([
    [1., 0., 0.],
    [0., -1., 0.],
    [0., 0., -1.]])
t_w_ned = np.array([0., 0., 0.])

# rotate from body to imu frame
R_b_i = np.array([
    [0., -1., 0.],
    [1., 0., 0.],
    [0., 0., 1.]])

NameError: name 'self' is not defined

In [10]:
from dynamicsSim import *
dyn = dynamics()
dt = rotor['Time'][1]-rotor['Time'][0]
i = 0
state = [x[0],y[0],z[0],0,0,0,mocap['pose.orientation.x'][i],mocap['pose.orientation.y'][i],mocap['pose.orientation.z'][i],mocap['pose.orientation.w'][i],0,0,0]
state2 = dyn.propogateDynamics(np.array(state),np.array([rotor['rpm_0'],rotor['rpm_1'],rotor['rpm_2'],rotor['rpm_3']]),dt)
print(np.round(state,3))
print(np.round(state2,3))
print(dt)

[[3.88195173e+03]
 [8.52834527e+00]
 [2.02919957e+05]]
[-3.484  0.385 -0.771  0.     0.     0.    -0.     0.01  -0.004  1.
  0.     0.     0.   ]
[ 9.951000e+00  4.140000e-01  7.015120e+02  3.881952e+03  0.000000e+00
  0.000000e+00  0.000000e+00  1.000000e-02 -4.000000e-03  1.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00]
0.0034608840942382812
